# Example File for Final Project

This notebook provides an example to generate a TRISO cylindrical compact placed in an hexagonal graphite matrix.  The data provided does not match the geometry of the project so this should be used as a guide only.

In [ ]:
%matplotlib inline
import openmc
import openmc.deplete

In [ ]:
fuel = openmc.Material(name='fuel')
fuel.add_nuclide('U238', 0.75, 'wo')
fuel.add_nuclide('U235', 0.14, 'wo')
fuel.add_nuclide('O16', 0.09, 'wo')
fuel.add_nuclide('C0', 0.02, 'wo')
fuel.set_density('g/cc', 10.5)

buff = openmc.Material(name='Buffer')
buff.set_density('g/cm3', 1.0)
buff.add_nuclide('C0', 1.0)

PyC1 = openmc.Material(name='PyC1')
PyC1.set_density('g/cm3', 1.0)
PyC1.add_nuclide('C0', 1.0)

PyC2 = openmc.Material(name='PyC2')
PyC2.set_density('g/cm3', 1.0)
PyC2.add_nuclide('C0', 1.0)

SiC = openmc.Material(name='SiC')
SiC.set_density('g/cm3', 3.0)
SiC.add_nuclide('C0', 0.5)
SiC.add_element('Si', 0.5)

In [ ]:
graphite = openmc.Material(name='moderator')
graphite.add_nuclide('C0',  1.0, 'wo')
graphite.set_density('g/cc', 1.65)

In [ ]:
materials_file = openmc.Materials([fuel, graphite, buff, PyC1, PyC2, SiC])
materials_file.cross_sections = '/home/shared/endfb71_hdf5/cross_sections.xml'
materials_file.export_to_xml()

This block defines a fuel radius and a slightly smaller compact radius to avoid situations where TRISOs would be cut by the boundary.

In [ ]:
top = openmc.ZPlane(surface_id=4, z0=+1, boundary_type='reflective')
bottom = openmc.ZPlane(surface_id=5, z0=-1, boundary_type='reflective') 

# Geometry definitions
fuel_radius = openmc.ZCylinder(surface_id=1, r=0.635) 
coolant_radius = openmc.ZCylinder(surface_id=2, r=0.35) 

compact_radius = openmc.ZCylinder(surface_id=6, r=0.63) 

top_compact = openmc.ZPlane(surface_id=7, z0=+0.999)
bottom_compact = openmc.ZPlane(surface_id=8, z0=-0.999) 

region = -compact_radius & -top_compact & +bottom_compact

fuel_region = -fuel_radius
moder_fuel = +fuel_radius
coolant_region = -coolant_radius 
moder_coolant = +coolant_radius 
 
fuel_cell = openmc.Cell(cell_id=1, fill=fuel, region=fuel_region)
moder_fuel_cell = openmc.Cell(cell_id=2, fill=graphite, region=moder_fuel)
graphite_cell = openmc.Cell(cell_id=3, fill=graphite)

This block creates a TRISO universe.

In [ ]:
# Create TRISO universe
spheres = [openmc.Sphere(r=1e-4*r)
           for r in [212.5, 312.5, 347.5, 382.5]]
cells = [openmc.Cell(fill=fuel, region=-spheres[0]),
         openmc.Cell(fill=buff, region=+spheres[0] & -spheres[1]),
         openmc.Cell(fill=PyC1, region=+spheres[1] & -spheres[2]),
         openmc.Cell(fill=SiC, region=+spheres[2] & -spheres[3]),
         openmc.Cell(fill=PyC2, region=+spheres[3])]
triso_univ = openmc.Universe(cells=cells)

This block places spheres of a given size in a region with a user defined packing fraction.  As the packing fraction gets larger, the run time increases substantially.

In [ ]:
outer_radius = 422.5*1e-4
centers = openmc.model.pack_spheres(radius=outer_radius, region=region, pf=0.15)

This block assigns a triso universe to each sphere.

In [ ]:
trisos = [openmc.model.TRISO(outer_radius, triso_univ, center) for center in centers]

This block creates a lattice over the TRISO region for acceleration purposes.  A virtual mesh is added and a list of TRISOs in each mesh cell is calculated.  Thus, when in a given cell, the nearest neighbor search is limited to surfaces in the mesh.

In [ ]:
lower_left, upper_right = fuel_cell.region.bounding_box
lower_left[2] = -1.0
upper_right[2] = 1.0
shape = (3, 3, 3)
pitch = (upper_right - lower_left)/shape
lattice = openmc.model.create_triso_lattice(
    trisos, lower_left, pitch, shape, graphite)

fuel_cell.fill = lattice

In [ ]:
fuel_u = openmc.Universe(universe_id=1001, cells=(fuel_cell,moder_fuel_cell))
graphite_u = openmc.Universe(universe_id=1002, cells=[graphite_cell])

In [ ]:
inner = [fuel_u]
outer = [fuel_u,fuel_u,fuel_u,fuel_u,fuel_u,fuel_u]

An hexagonal lattice is defined with a fuel universe surrounded by a graphite universe.

In [ ]:
hex_lat = openmc.HexLattice(lattice_id=1003, name='assembly')
hex_lat.center = (0., 0.)
hex_lat.pitch = (2.0,)
hex_lat.orientation = 'x'
hex_lat.outer = graphite_u
hex_lat.universes = [outer, inner]

An hexagonal bounding box is defined around the lattice.

In [ ]:
# Create the prism that will contain the lattice
outer_surface = openmc.model.hexagonal_prism(edge_length=3.5, orientation='x', boundary_type='reflective')

In [ ]:
# Fill a cell with the lattice. This cell is filled with the lattice and contained within the prism.
main_assembly = openmc.Cell(cell_id=7000, fill=hex_lat, region=outer_surface & -top & +bottom)

# Create a universe that contains both 
root = openmc.Universe(cells=[main_assembly])

In [ ]:
root.plot(origin = (0,0,0), pixels=(100, 100), width = (6.,6.), color_by = 'material')

In [ ]:
geom = openmc.Geometry(root)
geom.export_to_xml()

In [ ]:
# OpenMC simulation parameters

lower_left = [-3, -3, -1]
upper_right = [3, 3, 1]
uniform_dist = openmc.stats.Box(lower_left, upper_right, only_fissionable=True)
src = openmc.Source(space=uniform_dist)

settings = openmc.Settings()
settings.source = src
settings.batches = 100
settings.inactive = 25
settings.particles = 5000
settings.temperature = {'method': 'interpolation','range':(293.15,1923.15)}

settings.export_to_xml()

In [ ]:
openmc.run()

In [ ]:
fuel.volume = len(trisos)*4/3*3.1416*(212.5*1e-4)**3

In [ ]:
# Create depletion "operator"
#chain_file = 'chain_casl_pwr.xml'
#op = openmc.deplete.Operator(geometry=geom, settings=settings, chain_file=chain_file)

model = openmc.Model(geometry=geom, materials=materials_file, settings=settings)
#op = openmc.deplete.Operator(model, "/home/shared/endfb71_hdf5/chain_casl_pwr.xml")
op = openmc.deplete.Operator(model, "chain_casl_pwr.xml")

# Perform simulation using the predictor algorithm
time_steps = [1.0, 2.0, 10.0, 20.0, 30.0]  # days
power = 1000  
integrator = openmc.deplete.PredictorIntegrator(op, time_steps, power, timestep_units='d')
integrator.integrate()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Open results file
results = openmc.deplete.ResultsList.from_hdf5("depletion_results.h5")

# Obtain K_eff as a function of time
time, keff = results.get_eigenvalue()

# Obtain U235 concentration as a function of time
time, n_U235 = results.get_atoms('1', 'U235')

###############################################################################
#                            Generate plots
###############################################################################

days = 24*60*60
plt.figure()
plt.plot(time/days, keff[:,0], label="K-effective")
plt.xlabel("Time (days)")
plt.ylabel("Keff")
plt.show()

plt.figure()
plt.plot(time/days, n_U235, label="U235")
plt.xlabel("Time (days)")
plt.ylabel("n U5 (-)")
plt.show()
